In [189]:
#Creating the dataframes from CSV's of 4 celebrities

import numpy as np
import tensorflow as tf
import pandas as pd
biden = pd.read_csv("dataset/biden/real/"+"output.csv")
justin = pd.read_csv("dataset/justin/real/"+"output.csv")
#may = pd.read_csv("dataset/may/real/"+"output.csv")
pelosi = pd.read_csv("dataset/pelosi/real/"+"output.csv")
michelle = pd.read_csv("dataset/michelle/real/"+"output.csv")

biden['label'] = 0
justin['label'] = 1
#may['label'] = 'may'
pelosi['label'] = 2
michelle['label'] = 3

df_merged = pd.concat([biden, justin, michelle, pelosi])
Y = df_merged['label']
X = df_merged.drop('label', axis =1)

In [ ]:
# To incorporate for the timesteps 50, either we do some padding or get rid of some of the frames
X_new= X.iloc[0:610250]
Y = np.array(Y.iloc[0:610250])

In [191]:
#Normalizing the input
from sklearn.preprocessing import StandardScaler
X_new = StandardScaler().fit_transform(X_new)
norm_X = np.array(X_new)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [192]:
#Defining the inputs for the LSTM
timesteps = 50
data_dim = 426
num_classes = 4

In [193]:
#Reshaping the input matrix for the LSTM
updated_X = np.reshape(norm_X, (int(norm_X.shape[0]/timesteps), 50, 426))

In [194]:
#input shape for the LSTM
updated_X.shape

(12205, 50, 426)

In [203]:
#Preparing the target variable
i=0
count =0
target = []
for i in range(0,610249, 50):
    z = np.argmax(np.bincount(Y[i:i+50]))
    target.append(z)
    count = count+1
target = np.array(target)

In [206]:
#one hot encoding the target variable
encoder = LabelEncoder()
encoder.fit(target)
encoded_Y = encoder.transform(target)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [208]:
#imports for the LSTM model in Keras

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.layers import Dropout
from keras.callbacks import TensorBoard
from keras.models import load_model
from keras.layers import LSTM, Dense

In [209]:
# expected input data shape: (no of samples, timesteps, data_dim)
model = Sequential()
model.add(LSTM(426, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 426
model.add(LSTM(200, return_sequences=True))  # returns a sequence of vectors of dimension 200
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(4, activation='softmax'))

In [210]:
#compiling the LSTM model and the displaying the summary
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

In [223]:
#Splitting into train, test and validation
X_train, X_test, y_train, y_test = train_test_split(updated_X, dummy_y, test_size=0.33, random_state=42)

In [224]:
#fitting the data to the LSTM model and 

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)
model.fit(X_train, y_train,
          batch_size=64, validation_split=0.3, shuffle = True,  epochs=3, callbacks=[tensorboard])

Train on 5723 samples, validate on 2454 samples
Epoch 1/3
5723/5723 [==============================] - 149s 26ms/step - loss: 0.1462 - acc: 0.9614 - val_loss: 0.0666 - val_acc: 0.9772
Epoch 2/3
5723/5723 [==============================] - 102s 18ms/step - loss: 0.0437 - acc: 0.9892 - val_loss: 0.0160 - val_acc: 0.9955
Epoch 3/3
5723/5723 [==============================] - 102s 18ms/step - loss: 0.0196 - acc: 0.9942 - val_loss: 0.0047 - val_acc: 0.9992


In [225]:
#saving the model, deleting from the memory and restoring it again
model.save("lstm_facial.h5")
del model

model = load_model("lstm_facial.h5")

In [239]:
#predicting on the test dataset
y_pred = model.predict_classes(X_test)

In [245]:
#length of the predictions
len(y_pred)

4028

In [242]:

#for decoding the one hot encoded input of the original test set
lst = []
def decode(datum):
    return np.argmax(datum)

for i in range(y_test.shape[0]):
    lst.append(decode(y_test[i]))

In [251]:
#accuracy from the test set
accuracy = (y_pred == np.array(lst)).mean()
print(accuracy)

0.9985104270109235


In [1]:
#dropped_X = X.drop([X.columns[[1, 69]]], axis=1)